In [5]:
#!pip install nltk


In [ ]:

import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nmako\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
